一旦仮で作ったところまで

In [4]:
import pandas as pd

# データAとデータBのCSVファイルのパス
data_a_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/data/original_data/race_table/combined.csv'  # データAのCSVファイルパス
data_b_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/data/merged_data/merged_sorted.csv'  # データBのCSVファイルパス

# CSVファイルからデータを読み込む
data_a = pd.read_csv(data_a_path,low_memory=False)
data_b = pd.read_csv(data_b_path)
data_a=data_a.dropna()
data_b=data_b.dropna()
data_a=data_a.drop(columns=['first_horses', 'second_horses', 'third_horses', 'Time', 'Jockey', 'Trainer'])
# Code列でデータを結合 (data_bにdata_aの情報を追加)
merged_data = pd.merge(data_b, data_a, on='Code', how='left')

#結果の確認
merged_data.to_csv('../notebook/data-shin/merged_data_final.csv', index=False)
# 各列のユニークな値とその出現回数を取得
for i in merged_data.columns:
    value_counts = merged_data[i].value_counts()  # 各値とそのカウント
    print(f"{i}: {value_counts}")




/var/folders/0y/s3g9vbzd3vvch0mp2zlmz2pr0000gn/T/ipykernel_19076/1585316847.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_b = pd.read_csv(data_b_path)


Code: Code
201144020402    8
202009010109    8
201205040108    8
199808050206    8
199704030402    8
               ..
200054061909    1
200054070211    1
200054051509    1
200554082012    1
200143011012    1
Name: count, Length: 342407, dtype: int64
Rank: Rank
2       338972
3       338938
4       332252
5       321381
5         2640
3         2639
2         2636
4         2632
4.0        926
3.0        922
2.0        920
5.0        918
1          544
5(降)        23
4(降)         7
1            6
3(降)         3
1.0          2
失            1
Name: count, dtype: int64
Frame Rank: Frame Rank
8.0    209234
7.0    199961
6.0    181477
5.0    171365
4.0    154208
3.0    150563
2.0    142677
1.0    136877
Name: count, dtype: int64
Horse Number: Horse Number
2     128256
3     127873
4     127705
5     127133
6     126134
1     125458
7     120502
8     114465
9      97686
10     79712
11     56331
12     43213
13     24801
14     20448
15     13179
16     10318
17      1756
18      1392
Name:

なんかパスが奇妙なエラーを吐くので一旦絶対パス使ってます。

In [1]:
import matplotlib.pyplot as plt
print("matplotlib is successfully installed!")


matplotlib is successfully installed!


In [1]:
import pandas as pd
import numpy as np

# データの読み込み
file_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/notebook/data-shin/merged_data_final.csv'
df = pd.read_csv(file_path)

# 1. Sex/Ageを分割して新しい列に追加
df[['Sex', 'Age']] = df['Sex/Age'].str.extract(r'([A-Za-z]+)(\d+)')
df['Age'] = pd.to_numeric(df['Age'])  # 年齢は数値型に変換
df.drop(columns=['Sex/Age'], inplace=True)  # 元のSex/Age列を削除

# 2. カテゴリカルデータをOne-Hot Encoding
categorical_columns = ['Sex', 'Jockey', 'Track', 'Weather', 'Condition', 'Trainer', 'Banushi', 'Horse Name']

df = pd.get_dummies(df, columns=categorical_columns)

# 3. 欠損値の処理（欠損値を削除）
df.dropna(inplace=True)  # 欠損値を含む行を削除

# 4. 不要な列の削除（CodeやRankなど予測に不要なら削除）
columns_to_drop = ['Code', 'Rank', 'Frame Rank']  # 必要に応じて列を指定
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# 5. 前処理後のデータを新しいCSVファイルに保存
output_path = 'processed_horse_data.csv'  # 保存するファイル名を指定
df.to_csv(output_path, index=False)

print(f"前処理が完了しました。処理済みデータは {output_path} に保存されました。")


/var/folders/0y/s3g9vbzd3vvch0mp2zlmz2pr0000gn/T/ipykernel_22281/2957071519.py:6: DtypeWarning: Columns (1,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


: 

In [1]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
df=pd.read_csv('data')      #ここにデータファイルのパス
df_train, df_test = train_test_split(df, test_size=0.4)
x_train = df_train[['color_intensity']]
x_test  = df_test[['color_intensity']]

y_train = df_train['proline']
y_test  = df_test ['proline']
print(y_train)
lss = Lasso()
lss.fit(x_train, y_train)
y_pred = lss.predict(x_test)

plt.scatter(x_test, y_test)
x_for_plot = np.arange(np.min(x_test["color_intensity"]),np.max(x_test["color_intensity"]),0.1).reshape(-1,1)
plt.scatter(x_for_plot,lss.predict(x_for_plot),color="red")
plt.xlabel("color_intensity")
plt.ylabel("proline")
plt.show()

print(mean_squared_error(y_test,y_pred))

FileNotFoundError: [Errno 2] No such file or directory: 'data'